# Arbeidsbok: Nye stats

Du arbeider her for å videreføre arbeidet

Du har gjort:
- Beregnet steady-state course- og speed change
- Beregnet forholdet, ikke bare true/false når det kommer til dSOG og dCOG og hvor vidt de overstiger verdier for før-perioden (overshoot)
- Du har begrenset før- og etter-periodene til 60 sek før og etter ikke alle observasjoner før og etter
- Du har sørget for å bruke circmean istedenfor mean på sirkulære verdier (viktig)
- Du beregner avstanden mellom skipene idet de foretar sin manøver
- Du gjør utvelgelse ift lengde på skip, ikke med absolutt størrelse
- Du henter ut lats og lons for situasjonene, slik at du kan plotte dem

Du kan fremdeles:
- Ha en høyere begrensnning for hva det vil si å registreres som en speed/course maneuver (prøv deg frem med dette og plott i R)
- Leke deg rundt med om mean eller max er best for å bestemme course/speed maneuver
- NB: Husk å ta ut no action-ene, eventuelt bruke steady-state og total speed change / coruse change til å teste robustheten til disse
- Bruke mean rolling filter for steady course / speed change isteden
- Plotte dSOG, SOG osv for noen utvalgte situasjoner - se at ting stemmer

In [1]:
# Importing relevant libraries
import numpy as np
import pandas as pd 
import datetime as dt
import pyarrow.parquet as pq
import glob, os

from NearCollisionStatistics import *

# Setting internal options for speedup
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

pd.set_option('display.max_columns', 500)



# Reading small dataset from CSV

Reading the small one-day sample from CSV file. This does not take increadibly much memory (but still quite some).

In [3]:
### Building a databank ###

# Re-reading full AIS original file
PATH_csv = "~/code/DATA_DNV/ais_small.csv"

incols = ["date_time_utc","mmsi","lat","lon","sog","cog","true_heading","length","nav_status","RISK_Norwegian_Main_Vessel_Category_ID"]

AIS_df = pd.io.parsers.read_csv(PATH_csv
                        ,engine="c"
                        ,sep=";"
                        ,usecols=incols        
                        ,compression=None
                        ,dtype={"mmsi": np.uint64,             
                                "lat": np.float32, 
                                "lon": np.float32,
                                "sog": np.float32,
                                "cog": np.float32,
                                "true_heading": np.float32,
                                "length": np.float32,
                                "nav_status": np.int32,
                                "RISK_Norwegian_Main_Vessel_Category_ID": np.float32}
                        )

# Renaming fields
AIS_df.rename(columns={"date_time_utc": "Time",
                        "mmsi": "ID",
                        "lon": "LON",
                        "lat": "LAT",
                        "nav_status": "Status",
                        "true_heading": "Heading",
                        "sog": "SOG",
                        "cog": "COG",
                        "length": "Length",
                        "RISK_Norwegian_Main_Vessel_Category_ID": "Category"
                         },inplace=True)

# Converting time column from string to datetime
AIS_df["Time"] = pd.to_datetime(AIS_df.Time,format="%Y-%m-%dT%H:%M:%S.%f",box=False)

# Storing datetimes in the dataframe
AIS_df["Time_datetime"] = AIS_df["Time"]

# Converting datetimes to ints for later conversion to intervals
AIS_df["Time"] = AIS_df.Time.values.astype(np.uint64) / 10**9

# Removing platforms
AIS_df = AIS_df[(AIS_df.ID > 99999999)&(AIS_df.ID <= 999999999)]

# Sorting by time
AIS_df.sort_values(by = "Time",inplace=True)

## Computing on the small dataset

Computing happens here.

In [ ]:
AIS_filtered = time_to_CPA_calculator(AIS_df)

In [4]:
# Collecting observations
DATABANK = observation_collector(AIS_df, AIS_filtered)

AIS_databank = DATABANK

# Synchronizing times
databank_mergetime = observation_synchronizer(AIS_databank)

In [5]:
### Calculating Statistics (CPA, tCPA, CPA_dist) for all observations ###

AIS_db = databank_mergetime

# Calculating stats
AIS_with_stats = Stat_calculator(AIS_db)

# Calculating dSOG and dCOGs
AIS_with_diffs = diffs_computer(AIS_with_stats)


Removing 546 erroneous records
Removing 5572 inf entitites
Removing 0 NaN entitites


In [4]:
AIS_with_diffs = pd.io.parsers.read_csv("AIS_with_stats.csv")

In [10]:
### Calculating summary statistics ###
AIS2_db = AIS_with_diffs

# Calculating summart statistics
AIS_summary_stats = statistics_aggregator(AIS2_db)

AIS_with_diffs.to_csv("AIS_with_stats.csv",index=False)

# Writing final output for further analysis
AIS_summary_stats.to_csv("AIS_summary_stats.csv",index=False,
                     )

# Reading from parquet and writing to HDF5

Reading and writing to queryable format

In [2]:
files = glob.glob("/users/arnsteinvestre/code/Parquet/*.parquet")
cols = ["Timestamp", "MMSI-nummer", "Longitude", "Latitude", "Speed_over_ground", "Course_over_ground", "Length", "Navigational_status", "Risk_category","Width"]
pqdataset = pq.ParquetDataset(files)
AIS_pqds = pqdataset.read(columns = cols)
AIS_pd = AIS_pqds.to_pandas()

/anaconda/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
# Creating day labels
AIS_pd["Daystamp"] = AIS_pd["Timestamp"].dt.normalize()
AIS_pd["Day"] = AIS_pd["Daystamp"].factorize()[0]

In [4]:
# Renaming fields
AIS_pd.drop(labels="Daystamp",axis=1,inplace=True)

In [5]:
AIS_pd.rename(columns={"Timestamp": "Time",
                        "MMSI-nummer": "ID",
                        "Longitude": "LON",
                        "Latitude": "LAT",
                        "Navigational_status": "Status",
                        "Speed_over_ground": "SOG",
                        "Course_over_ground": "COG",
                        "Length": "Length",
                        "Risk_category": "Category"
                         },inplace=True)

In [6]:
# Storing datetimes in the dataframe
AIS_pd["Time_datetime"] = AIS_pd["Time"]

# Converting datetimes to ints for later conversion to intervals
AIS_pd["Time"] = AIS_pd.Time.values.astype(np.uint64) / 10**9

In [ ]:
#AIS_pd.drop(columns = ["IMO","Risk_category_name","Ship_cargo_type_ID"], inplace = True)
# If not properly read (with cols restriction)

In [13]:
data_store = pd.HDFStore("AIS_from_parquet_large.h5")

for day in range(0,12):

    AIS_daytable = AIS_pd[AIS_pd["Day"] == day]
    
    print("Day",day,"separated")
    
    data_store["day{}_df".format(day)] = AIS_daytable
    
    print("Day",day,"written")
    
data_store.close()

Day 0 separated
Day 0 written
Day 1 separated
Day 1 written
Day 2 separated
Day 2 written
Day 3 separated
Day 3 written
Day 4 separated
Day 4 written
Day 5 separated
Day 5 written
Day 6 separated
Day 6 written
Day 7 separated
Day 7 written
Day 8 separated
Day 8 written
Day 9 separated
Day 9 written
Day 10 separated
Day 10 written
Day 11 separated
Day 11 written


# Collecting instances on the large set

We now transition to reading the large file and turning this into a larger databank.
This requires using a larger part of memory, thus be careful.

## First some testing

We need to check that it works on a small sample

In [14]:
data_store = pd.HDFStore("AIS_from_parquet_large.h5")
AIS_pd = data_store["day1_df".format(day)]
data_store.close()

In [22]:
AIS_pd_samp = AIS_pd[np.random.rand(len(AIS_pd.index)) < 0.05].copy()

In [23]:
# Creating intervals
AIS_pd_samp["Interval"] = (AIS_pd_samp.Time - np.min(AIS_pd_samp.Time)) // 20
AIS_pd_samp["Interval"] = AIS_pd_samp.Interval.values.astype(np.uint32)

In [24]:
AIS_shorter = AIS_pd_samp.drop_duplicates(["ID","Interval"],keep="first")
AIS_shorter = AIS_shorter.sort_values(by=["Time","LON","LAT"], axis=0, inplace=False)

In [38]:
AIS_filtered = time_to_CPA_calculator(AIS_shorter,namedict = {"DateTime": "Time_datetime", 
                            "ID": "ID",
                            "Time": "Time",
                            "COG": "COG",
                            "SOG": "SOG",
                            "Heading": "Heading",
                            "LON": "LON",
                            "LAT": "LAT",
                            "Status": "Status",
                            "Length": "Length",
                            "Interval": "Interval",
                            "Distances": "Distances",
                            "Day": "Day"})

In [ ]:
AIS_collector = AIS_filtered

# Collecting observations
DATABANK_parq = observation_collector(AIS_pd_samp, AIS_filtered)

# Synchronizing times
databank_mergetime_l = observation_synchronizer(DATABANK_parq)
# Computing time to cpas and differentiations
AIS_large_with_stats = Stat_calculator(databank_mergetime_l)
AIS_large_with_diffs = diffs_computer(AIS_large_with_stats)

In [69]:
# Computing summary stats for each situation
AIS_large_summary_stats = statistics_aggregator(AIS_large_with_diffs)



## Now it starts for real
This is a loop that ideally computes for the whole set

In [4]:
### Read whole HDF5 table
data_store = pd.HDFStore("AIS_from_parquet_large.h5")
day1 = 0
AIS_collector_flag = False

data_store2 = pd.HDFStore("AIS_large_instances.h5")

for day in range(day1,12):
    
    AIS_pd = data_store["day{}_df".format(day)]
    
    AIS_pd["Time_Datetime"] = AIS_pd["Time_datetime"]
    AIS_pd.drop(columns=["Time_datetime"],inplace=True)
    # Removing platforms
    AIS_pd = AIS_pd[(AIS_pd.ID > 99999999)&(AIS_pd.ID <= 999999999)]

    # Creating intervals
    AIS_pd["Interval"] = (AIS_pd.Time - np.min(AIS_pd.Time)) // 20
    AIS_pd["Interval"] = AIS_pd.Interval.values.astype(np.uint32)
    
    AIS_shorter = AIS_pd.drop_duplicates(["ID","Interval"],keep="first")
    AIS_shorter = AIS_shorter.sort_values(by=["Time","LON","LAT"], axis=0, inplace=False)
    
    print("Starting calculations day",day)
    
    AIS_filtered = time_to_CPA_calculator(AIS_shorter)

    data_store2["Large_instances_day_{}".format(day)] = AIS_filtered
    
    if not AIS_collector_flag:
        AIS_collector = AIS_filtered
        AIS_collector_flag = True
    else:
        AIS_collector = pd.concat([AIS_collector, AIS_filtered])
    
    print("Finished day",day)
data_store.close()

print("Storing")
# Lagre mellomlagring
data_store2["Large_instances_all"] = AIS_collector
data_store2.close()



Starting calculations day 8
Finished day 8
Starting calculations day 9
Finished day 9
Starting calculations day 10
Finished day 10
Starting calculations day 11
Finished day 11


# Using instances to collect databank from large set

The databank is collected here

In [15]:
data_storeA = pd.HDFStore("AIS_large_istances_day0to7.h5")
data_storeB = pd.HDFStore("AIS_large_istances_8to11.h5")

AIS_large_filtered1 = pd.concat([data_storeA["Large_instances_day0to7"], data_storeB["Large_instances_8to11"]])
AIS_databank_flag = False

data_storeA.close()
data_storeB.close()

data_store = pd.HDFStore("AIS_from_parquet_large.h5")

data_storeSTAT = pd.HDFStore("AIS_summary_stats_large_backup.h5")

for day in range(0,12):
    AIS_pd = data_store["day{}_df".format(day)]
    
    # Collecting observations
    DATABANK_parq = observation_collector(AIS_pd, AIS_large_filtered1[AIS_large_filtered1["Day"] == day])
    
    # Synchronizing times
    databank_mergetime_l = observation_synchronizer(DATABANK_parq)
    
    # Computing time to cpas and differentiations
    AIS_large_with_stats = Stat_calculator(databank_mergetime_l)
    AIS_large_with_diffs = diffs_computer(AIS_large_with_stats)
    
    # Saving stat-ed file
    AIS_large_with_diffs.to_csv("AIS_large_with_stats_day{}.csv".format(day),
                                index=False)
    
    # Computing summary stats for each situation
    AIS_large_summary_stats = statistics_aggregator(AIS_large_with_diffs)

    data_storeSTAT["Stats_day_{}".format(day)] = AIS_large_summary_stats
    
    if not AIS_databank_flag:
        AIS_stats_databank_large = AIS_large_summary_stats
        AIS_databank_flag = True
    else:
        AIS_stats_databank_large = pd.concat([AIS_stats_databank_large,AIS_large_summary_stats])
    
    print("Finished day",day)

# Writing final output for further analysis
data_store.close()

AIS_stats_databank_large = AIS_stats_databank_large.reset_index(drop = True)
AIS_stats_databank_large["Situation_glob"] = AIS_stats_databank_large.index + 1

AIS_stats_databank_large.to_csv("AIS_large_summary_stats.csv",index=False,
                     )

Removing 1226 erroneous records
Removing 2887 inf entitites
Removing 0 NaN entitites
Finished day 0
Removing 1516 erroneous records
Removing 6637 inf entitites
Removing 0 NaN entitites
Finished day 1
Removing 1595 erroneous records
Removing 3591 inf entitites
Removing 0 NaN entitites
Finished day 2
Removing 120175 erroneous records
Removing 10407 inf entitites
Removing 0 NaN entitites
Finished day 3
Removing 724 erroneous records
Removing 3078 inf entitites
Removing 0 NaN entitites
Finished day 4
Removing 5619 erroneous records
Removing 4813 inf entitites
Removing 0 NaN entitites
Finished day 5
Removing 706 erroneous records
Removing 3366 inf entitites
Removing 0 NaN entitites
Finished day 6
Removing 1353 erroneous records
Removing 3948 inf entitites
Removing 0 NaN entitites
Finished day 7
Removing 4848 erroneous records
Removing 21607 inf entitites
Removing 0 NaN entitites
Finished day 8
Removing 863 erroneous records
Removing 2129 inf entitites
Removing 0 NaN entitites
Finished day 9

In [ ]:
a = 5

In [6]:
AIS_large_filtered1

,ID_Vessel_1,Time_Vessel_1,LON_Vessel_1,LAT_Vessel_1,SOG_Vessel_1,COG_Vessel_1,Length_Vessel_1,Status_Vessel_1,ID_Vessel_2,Time_Vessel_2,LON_Vessel_2,LAT_Vessel_2,SOG_Vessel_2,COG_Vessel_2,Length_Vessel_2,Status_Vessel_2,Min_time_to_CPA,CPA,Day
184568,244234000,2015-12-15 10:42:56,10.58720,59.11450,11.9,34.0,NaN,0,376185000,2015-12-15 10:42:45,10.57180,59.11740,10.3,217.4,NaN,0,0.004964,937.427612,0
268075,258509000,2015-12-15 14:52:45,10.69350,59.89460,9.8,62.2,NaN,0,258266500,2015-12-15 14:52:41,10.69180,59.89530,22.9,237.1,NaN,0,0.005133,122.921204,0
342169,259097000,2015-12-15 18:34:06,6.32020,62.39150,11.3,178.0,NaN,8,259098000,2015-12-15 18:34:06,6.32980,62.39160,9.4,356.0,NaN,0,0.018247,495.392181,0
239131,225424000,2015-12-15 13:33:27,10.62300,59.61810,14.5,176.3,NaN,0,311164000,2015-12-15 13:33:33,10.61630,59.61820,9.9,173.0,NaN,0,0.035673,377.408997,0
3525,259620000,2015-12-15 00:14:46,5.16130,60.26180,7.9,189.8,NaN,0,219375000,2015-12-15 00:14:51,5.15183,60.26420,13.7,206.0,NaN,0,0.054630,587.174133,0
334114,257244500,2015-12-15 18:10:16,9.77765,59.00690,7.8,129.1,NaN,0,232722000,2015-12-15 18:10:03,9.77767,59.00690,8.3,133.5,NaN,0,0.059300,1.147752,0
112562,518631000,2015-12-15 07:04:34,10.61850,59.84160,10.1,210.6,NaN,5,258306500,2015-12-15 07:04:22,10.60890,59.84390,29.0,219.0,NaN,0,0.077480,594.797363,0
4480,229061000,2015-12-15 00:18:01,5.28853,59.60930,13.5,28.9,NaN,0,259998000,2015-12-15 00:18:04,5.27609,59.61100,10.7,206.8,NaN,0,0.100848,725.688782,0
75157,257074700,2015-12-15 05:09:06,5.56225,59.77220,11.1,37.8,NaN,0,257022800,2015-12-15 05:09:18,5.54650,59.77550,12.5,221.0,NaN,0,0.101552,956.011108,0
150725,518730000,2015-12-15 09:02:40,10.55290,59.52940,8.3,53.4,NaN,0,518631000,2015-12-15 09:02:59,10.54870,59.53080,9.9,231.2,NaN,5,0.102567,283.719269,0


In [10]:
data_storeB["Large_instances_8to11"]

,ID_Vessel_1,Time_Vessel_1,LON_Vessel_1,LAT_Vessel_1,SOG_Vessel_1,COG_Vessel_1,Length_Vessel_1,Status_Vessel_1,ID_Vessel_2,Time_Vessel_2,LON_Vessel_2,LAT_Vessel_2,SOG_Vessel_2,COG_Vessel_2,Length_Vessel_2,Status_Vessel_2,Min_time_to_CPA,CPA,Day
139175,257244500,2015-05-15 08:38:19,9.66566,59.05430,8.9,102.0,NaN,0,232722000,2015-05-15 08:38:11,9.66568,59.05430,9.0,104.7,NaN,0,0.000635,1.146426,8
299050,219347000,2015-05-15 16:43:40,5.54222,59.02150,23.4,175.5,NaN,0,231711000,2015-05-15 16:43:47,5.55024,59.02160,15.6,359.9,NaN,0,0.013957,459.659302,8
120775,258327500,2015-05-15 07:40:56,7.09780,62.68130,11.7,7.0,NaN,0,258328500,2015-05-15 07:40:56,7.09120,62.68150,10.9,189.0,NaN,0,0.037310,337.904327,8
292341,259436000,2015-05-15 16:22:58,10.18460,64.19870,7.8,30.1,NaN,0,258422000,2015-05-15 16:22:53,10.17960,64.19930,8.6,214.4,NaN,0,0.038384,251.383102,8
159523,245432000,2015-05-15 09:48:53,11.97530,65.28770,11.3,23.3,NaN,0,215349000,2015-05-15 09:48:41,11.96880,65.28820,14.2,204.0,NaN,0,0.045576,307.658844,8
274937,259638000,2015-05-15 15:27:55,10.18850,63.46980,11.6,153.1,NaN,0,257013700,2015-05-15 15:27:56,10.19440,63.47030,10.4,340.9,NaN,0,0.052287,298.563995,8
289611,257338000,2015-05-15 16:14:01,5.89015,59.13590,26.1,129.0,NaN,0,259592000,2015-05-15 16:14:02,5.89682,59.13880,9.2,144.0,NaN,0,0.054271,499.510254,8
149128,311571000,2015-05-15 09:13:31,4.64067,60.85930,10.7,269.0,NaN,0,258196000,2015-05-15 09:13:38,4.64116,60.85930,10.8,262.3,NaN,0,0.067267,26.572075,8
86634,259592000,2015-05-15 05:56:37,5.89508,59.13980,11.5,154.0,NaN,0,257338000,2015-05-15 05:56:22,5.89614,59.13990,15.5,342.5,NaN,0,0.070827,61.519226,8
119733,259344000,2015-05-15 07:37:38,10.64100,59.41430,12.0,44.0,NaN,0,259402000,2015-05-15 07:37:37,10.63640,59.41550,13.5,226.0,NaN,0,0.079858,292.859680,8


In [8]:
data_storeA = pd.HDFStore("AIS_large_istances_day0to7.h5")
data_storeB = pd.HDFStore("AIS_large_istances_8to11.h5")

